In [ ]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 100.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 105.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 6.5 MB/s eta 0:00:00


In [ ]:
!pip install pyngrok

In [ ]:
!pip install ngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 77.8 MB/s eta 0:00:00


In [ ]:
!pip install rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.1/33.1 MB 51.2 MB/s eta 0:00:00


In [ ]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 60.9 MB/s eta 0:00:00


In [ ]:
!pip install torch

In [ ]:
# Remove non-printable characters from the code
with open('/content/app.py', 'w') as f:
    code = """
import streamlit as st
import torch
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GATConv, global_mean_pool
from rdkit import Chem
from rdkit.Chem import Draw
from transformers import RobertaTokenizer, RobertaModel

# Initialize GAT model
class GAT(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GAT, self).__init__()
        self.gat1 = GATConv(input_dim, hidden_dim, heads=4, concat=True)
        self.gat2 = GATConv(hidden_dim * 4, hidden_dim, heads=1, concat=True)
        self.fc = torch.nn.Linear(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.gat1(x, edge_index))
        x = F.relu(self.gat2(x, edge_index))
        x = global_mean_pool(x, data.batch)
        out = self.fc(x)
        return out

# Load the entire models
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the saved models directly
logp_qed_model = torch.load("logp_qed_model_complete.pth", map_location=device)
sas_model = torch.load("sas_model_complete.pth", map_location=device)

# Set models to evaluation mode
logp_qed_model.eval()
sas_model.eval()

# Load the ChemBERTa tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained('seyonec/ChemBERTa-zinc-base-v1')
chemberta_model = RobertaModel.from_pretrained('seyonec/ChemBERTa-zinc-base-v1').to(device)

# Function to convert SMILES to ChemBERTa embedding
def smiles_to_chemberta_embedding(smiles):
    inputs = tokenizer(smiles, return_tensors='pt', padding=True, truncation=True).to(device)
    with torch.no_grad():
        outputs = chemberta_model(**inputs)
    return outputs.last_hidden_state.mean(dim=1)

# Function to convert SMILES to graph
def smiles_to_graph(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        st.error("Failed to parse SMILES string. Please check the input.")
        return None

    atom_features = [atom.GetAtomicNum() for atom in mol.GetAtoms()]
    edge_index = []
    edge_attr = []

    for bond in mol.GetBonds():
        i = bond.GetBeginAtomIdx()
        j = bond.GetEndAtomIdx()
        edge_index.append((i, j))
        edge_index.append((j, i))  # Undirected graph
        edge_type = bond.GetBondTypeAsDouble()
        edge_attr.append(edge_type)
        edge_attr.append(edge_type)

    edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
    x = torch.tensor(atom_features, dtype=torch.float).view(-1, 1)
    edge_attr = torch.tensor(edge_attr, dtype=torch.float).view(-1, 1)

    return Data(x=x, edge_index=edge_index, edge_attr=edge_attr)

# Streamlit UI
st.title("Molecular Property Predictor")

# Input for SMILES string
smiles = st.text_input("Enter a SMILES string")

# Dropdown for selecting the property to predict
property_choice = st.selectbox("Select the property to predict", ("LogP/QED", "SAS"))

# Predict button
if st.button("Predict"):
    if smiles:
        # Convert SMILES to graph and embeddings
        chemberta_embedding = smiles_to_chemberta_embedding(smiles)
        graph = smiles_to_graph(smiles)

        if graph:
            # Move graph tensors to the correct device
            graph.x = graph.x.to(device)
            graph.edge_index = graph.edge_index.to(device)
            graph.edge_attr = graph.edge_attr.to(device)
            graph.batch = torch.zeros(graph.x.size(0), dtype=torch.long).to(device)

            # Concatenate embeddings with graph features
            repeated_embedding = chemberta_embedding.repeat(graph.x.size(0), 1)
            graph.x = torch.cat([graph.x, repeated_embedding], dim=1)

            if property_choice == "LogP/QED":
                with torch.no_grad():
                    prediction = logp_qed_model(graph).cpu().numpy()
                    st.write(f"Predicted logP: {prediction[0][0]:.4f}, QED: {prediction[0][1]:.4f}")
            else:
                with torch.no_grad():
                    prediction = sas_model(graph).cpu().numpy()
                    st.write(f"Predicted SAS: {prediction[0][0]:.4f}")

            # Display molecular image
            mol = Chem.MolFromSmiles(smiles)
            img = Draw.MolToImage(mol)
            st.image(img, caption="Molecular Structure")
        else:
            st.write("Invalid SMILES string.")
    else:
        st.write("Please enter a SMILES string.")



"""
    # Write clean code to the file
    f.write(code.replace('\xa0', ' '))  # Remove non-breaking spaces


In [ ]:
# Replace 'your_ngrok_token' with the actual token from ngrok dashboard
!ngrok authtoken 2nbmaoZ8rpR6l2OsISIbv1ezsyX_54n2FitHfB3N8EWisVgbo


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from pyngrok import ngrok

# Kill any previous tunnels, if running
ngrok.kill()

# Start the ngrok tunnel with the updated syntax for ngrok v3+
public_url = ngrok.connect(addr="8501")
print(f"Your Streamlit app is available at: {public_url}")

# Run the Streamlit app
!streamlit run app.py &>/dev/null&


Your Streamlit app is available at: NgrokTunnel: "https://ed88-34-143-177-2.ngrok-free.app" -> "http://localhost:8501"
